In [ ]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")
# Invoke a primitive. For more details see https://qiskit.org/documentation/partners/qiskit_ibm_runtime/tutorials.html
# result = Sampler("ibmq_qasm_simulator").run(circuits).result()

In [9]:
import numpy as np
from qiskit import QuantumCircuit, Aer, execute
import copy

k = 1 
n = 3  
s = n - k  
perr_x = 0  
perr_y = 0
perr_z = 0
shots = 1000  
trials =5 

def create_error_circuit(ax,ay):
    qc = QuantumCircuit(k, k)
    qc.rx(ax, range(k))
    qc.ry(ay, range(k))
    qc.measure(range(k), range(k))
    return qc


def create_encoding_circuit(ax,ay):
    qc=QuantumCircuit(n+s,k)
    qc.rx(ax, range(k))
    qc.ry(ay, range(k))
    qc.barrier()

    
    for i in range(k,n):
        qc.cx(i%k, i)

    return qc


def create_syndrome_circuit(qc):
   
    
    return qc

def create_decoding_circuit(qc):
    
    
    for i in range(k,n):
        qc.cx(i%k,i)
    qc.barrier()
    qc.measure(range(k), range(k))
    return qc


best_penalty=float('inf')
best_syndrome_circuit=None
penalties=[]  

for trial in range(trials):
    
    ax =np.random.rand()*2*np.pi
    ay =np.random.rand()*2*np.pi
    
    
    error_circuit = create_error_circuit(ax, ay)
    simulator = Aer.get_backend('statevector_simulator')
    result = execute(error_circuit, simulator, shots=shots).result()
    m1 = result.get_counts(error_circuit)
    
   
    encoding_circuit=create_encoding_circuit(ax, ay)
    
   
    for syndrome_trial in range(2 ** s):
        ec=copy.deepcopy(encoding_circuit)
        syndrome_circuit = create_syndrome_circuit(ec)
        for i in range(n):
            px = np.random.rand()
            py = np.random.rand()
            pz = np.random.rand()
        
            
            error_indices = []
            if px<perr_x:
                syndrome_circuit.x(i)
            if py<perr_y:
                syndrome_circuit.y(i)
            if pz<perr_z:
                syndrome_circuit.z(i)
            
        
        
        decoding_circuit=create_decoding_circuit(syndrome_circuit)
        
      
        result = execute(decoding_circuit, simulator, shots=shots).result()
        m2=result.get_counts(decoding_circuit)
        
        
        penalty=sum(abs(m1.get(key, 0)- m2.get(key, 0)) for key in set(m1) | set(m2))
        penalties.append(penalty)
        
        
        if penalty<best_penalty:
            best_penalty=penalty
            best_syndrome_circuit=syndrome_circuit
        

print(penalties)
print("Best Penalty:",best_penalty)
print("Best Syndrome Circuit:",best_syndrome_circuit)


[20, 42, 60, 112, 32, 12, 40, 64, 6, 44, 44, 42, 6, 32, 28, 2, 70, 18, 12, 30]
Best Penalty: 2
Best Syndrome Circuit:      ┌────────────┐┌────────────┐ ░                      ░ ┌─┐
q_0: ┤ Rx(4.9011) ├┤ Ry(5.2323) ├─░───■────■────■────■───░─┤M├
     └────────────┘└────────────┘ ░ ┌─┴─┐  │  ┌─┴─┐  │   ░ └╥┘
q_1: ─────────────────────────────░─┤ X ├──┼──┤ X ├──┼───░──╫─
                                  ░ └───┘┌─┴─┐└───┘┌─┴─┐ ░  ║ 
q_2: ─────────────────────────────░──────┤ X ├─────┤ X ├─░──╫─
                                  ░      └───┘     └───┘ ░  ║ 
q_3: ─────────────────────────────░──────────────────────░──╫─
                                  ░                      ░  ║ 
q_4: ─────────────────────────────░──────────────────────░──╫─
                                  ░                      ░  ║ 
  c: ═══════════════════════════════════════════════════════╩═
                                                              
